In [ ]:
import threading
from math import isqrt

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    limit = isqrt(n)
    for i in range(3, limit + 1, 2):
        if n % i == 0:
            return False
    return True

def prime_worker(start: int, end: int, out: list):
    for num in range(start, end):
        if is_prime(num):
            out.append(num)

def threaded_prime_check(low: int, high: int, threads: int = 4):
    span = (high - low) // threads
    threads_list, results = [], []
    for i in range(threads):
        s = low + i * span
        e = high if i == threads - 1 else s + span
        t = threading.Thread(target=prime_worker, args=(s, e, results))
        t.start()
        threads_list.append(t)
    for t in threads_list:
        t.join()
    results.sort()
    return results

if __name__ == "__main__":
    primes = threaded_prime_check(1, 100_000, threads=8)
    print(primes)

In [ ]:
import threading
from collections import Counter
from pathlib import Path

def count_words(lines: list[str], counter: Counter, lock: threading.Lock):
    local = Counter()
    for line in lines:
        words = [w.strip(".,;:!?\"'()[]").lower() for w in line.split()]
        local.update(words)
    with lock:
        counter.update(local)

def threaded_word_count(file_path: str, threads: int = 4) -> Counter:
    text = Path(file_path).read_text(encoding="utf-8").splitlines()
    chunk = len(text) // threads
    counters, lock = Counter(), threading.Lock()
    threads_list = []
    for i in range(threads):
        start = i * chunk
        end = len(text) if i == threads - 1 else start + chunk
        t = threading.Thread(target=count_words, args=(text[start:end], counters, lock))
        t.start()
        threads_list.append(t)
    for t in threads_list:
        t.join()
    return counters

if __name__ == "__main__":
    result = threaded_word_count("large_text.txt", threads=8)
    for word, freq in result.most_common(20):
        print(f"{word}: {freq}")






